In [24]:
import random
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from collections import defaultdict

Point = namedtuple('Point', ['x', 'y'])
Segment = namedtuple('Segment', ['p1', 'p2', 'id'])

class BentleyOttmann:
    def __init__(self):
        self.pq = []
        self.sl = []
        self.intersections = []
        self.segments = []
        self.segment_counter = 0
        self.intersecting_segments = defaultdict(list)  # Тут будем хранить пересекающиеся отрезки

    def add_segment(self, p1, p2):
        if p1.x > p2.x:
            p1, p2 = p2, p1
        
        segment = Segment(p1, p2, self.segment_counter)
        self.segment_counter += 1
        self.segments.append(segment)
        
        heapq.heappush(self.pq, Event(p1.x, 'left', segment))
        heapq.heappush(self.pq, Event(p2.x, 'right', segment))

    def find_intersections(self):
        while self.pq:
            event = heapq.heappop(self.pq)
            
            if event.type == 'left':
                self._handle_left_event(event)
            elif event.type == 'right':
                self._handle_right_event(event)
            elif event.type == 'intersection':
                self._handle_intersection_event(event)
        
        return self.intersections

    def _handle_intersection_event(self, event):
        seg1 = event.segment
        seg2 = event.other_segment
        point = event.intersection_point
        
        self.intersections.append(point)
        
        # Добавляем отрезки в список пересекающихся
        self.intersecting_segments[seg1.id].append(point)
        self.intersecting_segments[seg2.id].append(point)
        
        idx1 = self._find_index_in_sl(seg1)
        idx2 = self._find_index_in_sl(seg2)
        
        if abs(idx1 - idx2) != 1:
            return
        
        self.sl[idx1], self.sl[idx2] = self.sl[idx2], self.sl[idx1]
        
        new_above = self.sl[idx2+1] if idx2+1 < len(self.sl) else None
        new_below = self.sl[idx1-1] if idx1-1 >= 0 else None
        
        if new_above:
            self._check_intersection(seg2, new_above)
        if new_below:
            self._check_intersection(seg1, new_below)

    # Остальные методы остаются без изменений...

def visualize_segments(bo, segments):
    """Визуализация с использованием данных из алгоритма"""
    # Разделяем отрезки на пересекающиеся и нет
    intersecting = [s for s in segments if s.id in bo.intersecting_segments]
    non_intersecting = [s for s in segments if s.id not in bo.intersecting_segments]
    
    # Создаем фигуру
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Рисуем непересекающиеся отрезки
    if non_intersecting:
        lines = [[(s.p1.x, s.p1.y), (s.p2.x, s.p2.y)] for s in non_intersecting]
        lc = LineCollection(lines, colors='blue', linewidths=2)
        ax.add_collection(lc)
    
    # Рисуем пересекающиеся отрезки
    if intersecting:
        lines = [[(s.p1.x, s.p1.y), (s.p2.x, s.p2.y)] for s in intersecting]
        lc = LineCollection(lines, colors='red', linewidths=3)
        ax.add_collection(lc)
    
    # Рисуем точки пересечения
    if bo.intersections:
        x = [p.x for p in bo.intersections]
        y = [p.y for p in bo.intersections]
        ax.scatter(x, y, color='green', s=100, zorder=3)
    
    # Настройки графика
    ax.autoscale()
    ax.set_title(f'Segments Visualization ({len(bo.intersections)} intersections)')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.grid(True)
    
    plt.show()

# Генерация отрезков с гарантией пересечений
def generate_segments(n, width=100, height=100):
    segments = []
    
    # Гарантированные пересекающиеся пары
    segments.append(Segment(Point(30, 30), Point(70, 70), 0))
    segments.append(Segment(Point(30, 70), Point(70, 30), 1))
    segments.append(Segment(Point(20, 50), Point(80, 50), 2))
    segments.append(Segment(Point(50, 20), Point(50, 80), 3))
    
    # Случайные отрезки
    for i in range(4, n):
        x1 = random.randint(0, width)
        y1 = random.randint(0, height)
        x2 = random.randint(0, width)
        y2 = random.randint(0, height)
        segments.append(Segment(Point(x1, y1), Point(x2, y2), i))
    
    return segments

# Основной код
n = 15  # Количество отрезков
segments = generate_segments(n)

bo = BentleyOttmann()
for seg in segments:
    bo.add_segment(seg.p1, seg.p2)

intersections = bo.find_intersections()
print(f"Найдено {len(intersections)} точек пересечения")

visualize_segments(bo, segments)

AttributeError: 'BentleyOttmann' object has no attribute '_handle_left_event'